In [0]:
from google.colab import drive
import os
import h5py
from keras.preprocessing.image import load_img,img_to_array
from keras.layers import Input, Dense, Conv2D, MaxPool2D, UpSampling2D, concatenate,Dropout,MaxPooling2D,Conv2DTranspose
from keras.models import Model
import keras
import numpy as np
from keras.models import load_model
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping
import gc
import tensorflow as tf
from keras import backend as K
from sklearn import preprocessing as pp
from keras import layers, activations
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
import cv2
from skimage.util.shape import view_as_windows
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [0]:

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
os.chdir("/content/gdrive/My Drive/aula_sem_seg")
!ls

 1_1.png					 Reference_Test.tif
 1.png						 Reference_Train.tif
 2_2.png					 semantic_segmentaion.ipynb
 2.png						'SemSeg (1).ipynb'
 ddpg_pendulum_exercise.ipynb			'SemSeg (9).ipynb'
 example_seg.ipynb				 SemSeg.ipynb
 gt						 SemSeg.pptx
 images						 t1.png
 Images						 t2.png
 Image_Test.tif					 train_gt.png
 Image_Train.tif				 train.png
 img_1_32.png					 Transfer_Learning.ipynb
 img_reconstruct_128.png			 weed_1_32.png
 img_reconstruct_164.png			 weed_2_32.png
 img_reconstruct_256.png			 weed_3_32.png
 img_reconstruct_32.png				 weed_4_32.png
 img_reconstruct_remote_sensing_model1_256.png	 weed_5_32.png
 img_reconstruct_weed_1_264.png			 weed_j_32.png
 img_reconstruct_weed_164.png			 weed_r_32.png
 m1.png						 weed_result1_128.png
 m2.tif						 weed_result1_256.png
 model_a.h5					 weed_result_128.png
 model_a.json					 weed_result1_64.png
 model_b.h5					 weed_result_32.png
 model.json					 weed_result_64.png
 proyecto_dl_2019.ipynb				 weed_will.ipynb
 proyecto.ipynb

In [0]:
def weights_categorical_crossentropy(weights):
   
    if isinstance(weights,list) or isinstance(np.ndarray):
        weights=K.variable(weights)

    def loss(target,output,from_logits=False):
        if not from_logits:
            output /= tf.reduce_sum(output,
                                    len(output.get_shape()) - 1,
                                    True)
            _epsilon = tf.convert_to_tensor(K.epsilon(), dtype=output.dtype.base_dtype)
            output = tf.clip_by_value(output, _epsilon, 1. - _epsilon)
            weighted_losses = target * tf.log(output) * weights
            return - tf.reduce_sum(weighted_losses,len(output.get_shape()) - 1)
        else:
            raise ValueError('error')
    return loss
def read_img_rgb(img_path):
	img = cv2.imread(img_path)
	return img


def read_img_gray(img_path):
	img = cv2.imread(img_path,0)
	return img


def show_result(acc, val_acc, loss, val_loss):
    plt.rcParams['axes.facecolor']='white'
    f, axarr = plt.subplots(1 , 2)
    f.set_figwidth(10)

    # Accuracy
    axarr[0].plot(acc)
    axarr[0].plot(val_acc)
    axarr[0].set_title('model accuracy')
    axarr[0].set_ylabel('accuracy')
    axarr[0].set_xlabel('epoch')
    axarr[0].legend(['train', 'valid'], loc='upper left')

    # Loss
    axarr[1].plot(loss)
    axarr[1].plot(val_loss)
    axarr[1].set_title('model loss')
    axarr[1].set_ylabel('loss')
    axarr[1].set_xlabel('epoch')
    axarr[1].legend(['train', 'valid'], loc='upper left')
    
    plt.show()

def extract_patches(img_array_path, img_reference_path, patches_size, stride):
  
    """
    Funtion to extract subimages patches 
    """
    img_array = read_img_rgb(img_array_path)
    img_reference = read_img_gray(img_reference_path)
    
    rows = int(patches_size)
    unique_class = np.unique(img_reference)
    # convert from Gray to 0,1,2,3,4 classes 
    for i in range(len(unique_class)):
        img_reference[img_reference==unique_class[i]] = i

    # define the window_shape for patch extraction
    window_shape_array = (rows, rows, img_array.shape[2])
    window_shape_ref = (rows, rows)

    # ectract patches
    patches_array = np.array(view_as_windows(img_array, window_shape_array, step = stride))
    patches_ref = np.array(view_as_windows(img_reference, window_shape_ref, step = stride))
   
    k1,k2,p,row,col,depth = patches_array.shape

    patches_array = patches_array.reshape(k1*k2,row,col,depth)   
    patches_ref = patches_ref.reshape(k1*k2,row,col) 

    return patches_array, patches_ref
  

def data_augmentation(data, labels):
    """
    Funtion for data augmentation, for each patch apply rotation (90, 180, 270)
    and flip horizontal and vertical
    """
    augmented_data = data
    augmented_labels = labels


    cont_transf = 0
    for i in range(6):                
        augmented_data_temp = data
        augmented_label_temp = labels
        
        if cont_transf == 0:
            augmented_data_temp = np.rot90(augmented_data_temp,1,(1,2))
            augmented_label_temp = np.rot90(augmented_label_temp,1,(1,2))
        
        elif cont_transf == 1:
            augmented_data_temp = np.rot90(augmented_data_temp,2,(1,2))
            augmented_label_temp = np.rot90(augmented_label_temp,2,(1,2))

        elif cont_transf == 2:
            augmented_data_temp = np.flip(augmented_data_temp,1)
            augmented_label_temp = np.flip(augmented_label_temp,1)
            
        elif cont_transf == 3:
            augmented_data_temp = np.flip(augmented_data_temp,2)
            augmented_label_temp = np.flip(augmented_label_temp,2)
        
        elif cont_transf == 4:
            augmented_data_temp = np.rot90(augmented_data_temp,3,(1,2))
            augmented_label_temp = np.rot90(augmented_label_temp,3,(1,2))
            
        elif cont_transf == 5:
            augmented_data_temp = augmented_data_temp
            augmented_label_temp = augmented_label_temp
            
           
        augmented_data = np.vstack((augmented_data,augmented_data_temp))
        augmented_labels = np.vstack((augmented_labels,augmented_label_temp))

    idx = np.random.permutation(augmented_data.shape[0])
    out_data = augmented_data[idx]
    out_labels = augmented_labels[idx]

    return out_data, out_labels
  
  
def unblockshaped(blocks, h, w):
    """
    Funtion to mosaicking back the patches 
    """
    n, nrows, ncols = blocks.shape
    bpc = w//ncols
    bpr = h//nrows

    reconstructed = np.zeros((h,w))
    t = 0
    for i in range(bpr):
        for j in range(bpc):
            reconstructed[i*nrows:i*nrows+nrows,j*ncols:j*ncols+ncols] = blocks[t]
            t = t+1
    return reconstructed
  
def gray2rgb(image):
    """
    Funtion to convert classes values from 0,1,3,4 to rgb values
    """
    row,col = image.shape
    image = image.reshape((row*col))
    rgb_output = np.zeros((row*col, 3))
    rgb_map = [[0,0,255],[0,255,0],[0,255,255],[255,255,0],[255,255,255]]
    for j in np.unique(image):
        rgb_output[image==j] = np.array(rgb_map[j])
    
    rgb_output = rgb_output.reshape((row,col,3))  
    rgb_output = cv2.cvtColor(rgb_output.astype('uint8'),cv2.COLOR_BGR2RGB)
    return rgb_output
  
def compute_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1score = 100*f1_score(true_labels, predicted_labels, average=None)
    recall = 100*recall_score(true_labels, predicted_labels, average=None)
    prescision = 100*precision_score(true_labels, predicted_labels, average=None)
    return accuracy, f1score, recall, prescision

In [0]:
def EncDec (nClasses , optimizer=None , input_width=32 , input_height=32 , nChannels=3 ): 
  ########################################
  ##            Code here               ##
  ##  Change network architecture       ##
  ########################################    
  inputs = Input((input_height, input_width, nChannels))
  c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (inputs)
  c1 = Dropout(0.1) (c1)
  c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
  p1 = MaxPooling2D((2, 2)) (c1)

  c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
  c2 = Dropout(0.1) (c2)
  c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
  p2 = MaxPooling2D((2, 2)) (c2)

  c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
  c3 = Dropout(0.2) (c3)
  c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
  p3 = MaxPooling2D((2, 2)) (c3)

  c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
  c4 = Dropout(0.2) (c4)
  c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
  p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

  c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
  c5 = Dropout(0.3) (c5)
  c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

  u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
  u6 = concatenate([u6, c4])
  c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
  c6 = Dropout(0.2) (c6)
  c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

  u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
  u7 = concatenate([u7, c3])
  c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
  c7 = Dropout(0.2) (c7)
  c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

  u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
  c8 = Dropout(0.1) (c8)
  c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

  u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
  u9 = concatenate([u9, c1], axis=3)
  c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
  c9 = Dropout(0.1) (c9)
  c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

  
  outputs = Conv2D(64, (1, 1), activation='relu') (c9)

  outputs = Conv2D(3, (1, 1), activation='softmax') (outputs)
 

  model = Model(inputs, outputs)
  #custom_loss = weights_categorical_crossentropy(categorical_weight)

  model.compile(loss="categorical_crossentropy", optimizer=optimizer , metrics=['accuracy'])

  return model

#train and test
---


In [0]:
def Train_Test(net, train_array, train_ref, test_array, test_ref, batch_size, epochs):
    
    # apply data augmentation
    train_array,train_ref = data_augmentation(train_array,train_ref)
    
    # one_hot classes
    num_classes = len(np.unique(train_ref))
    imgs_mask_cat = train_ref.reshape(train_ref.shape[0]*train_ref.shape[1]*train_ref.shape[2])
    imgs_mask_cat = keras.utils.to_categorical(imgs_mask_cat, num_classes)
    train_ref = imgs_mask_cat.reshape(train_ref.shape[0],train_ref.shape[1],train_ref.shape[2],num_classes)
    imgs_mask_cat = []
    
    print('Start the training')
    patch_size = train_ref.shape[1]

    # train
    history = net.fit(train_array, train_ref,
                      batch_size = batch_size,
                      epochs = epochs,
                      verbose = 1,
                      shuffle = True)


    train_array, train_ref = 0,0 
    # clear memory 
    gc.collect() 

    # one_hot classes
    imgs_mask_cat = test_ref.reshape(test_ref.shape[0]*test_ref.shape[1]*test_ref.shape[2])
    imgs_mask_cat = keras.utils.to_categorical(imgs_mask_cat, num_classes)
    test_encoding = imgs_mask_cat.reshape(test_ref.shape[0],test_ref.shape[1],test_ref.shape[2],num_classes)
    imgs_mask_cat = []

    # evaluate in test
    score = net.evaluate(test_array, test_encoding, verbose=1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1]*100)
    
    return net

#Main 
---

In [0]:
if __name__=='__main__':
    
  # patches size
  ##### change here #####
  patches_size = 64

  # path for test and train data
  train_array_path = './train.png'
  train_reference_path = './train_gt.png'    
  test_array_path = './images/2.png'
  test_reference_path = './gt/2.png'


  # non overlapping patches with stride = patches_size
  stride = int(0.5*patches_size)
  weed_pixel = 0
  crop_pixel = 0
  background_pixel = 0
  # extract training pacthes 
  train_array,train_ref = extract_patches(train_array_path,
                                          train_reference_path, 
                                          patches_size, stride)
  
  print(train_ref[1].shape)
  numb_patch, row, col, bands = train_array.shape

  # extract test pacthes 
  test_array, test_ref = extract_patches(test_array_path,
                                         test_reference_path,
                                         patches_size, patches_size)

  # standarize features removing the mean and scaling to unit variance
  row,image_size,image_size,bands= train_array.shape            
  train_array = train_array.reshape(row,image_size*image_size*bands)            
  scaler = pp.StandardScaler().fit(train_array)            
  train_array = scaler.transform(train_array)
  train_array = train_array.reshape(row,image_size,image_size,bands)

  row,image_size,image_size,bands= test_array.shape            
  test_array = test_array.reshape(row,image_size*image_size*bands)                   
  test_array = scaler.transform(test_array)
  test_array = test_array.reshape(row,image_size,image_size,bands)

  # define model 
  optimazer = keras.optimizers.Adamax(lr = 1e-3)
  classes = 3      
  train_ref_forclassw = train_ref.reshape(train_ref.shape[0]*image_size*image_size)
  net = EncDec(classes, optimizer=optimazer, input_width=patches_size, input_height=patches_size, nChannels=bands)
  net.summary()

(64, 64)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 64, 64, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
dropout_19 (Dropout)            (None, 64, 64, 16)   0           conv2d_41[0][0]                  
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 64, 64, 16)   2320        dropout_19[0][0]                 
__________________________________________________________________________________________________
m

In [0]:

  # Train and Test
  #Call the train_test function
  model = Train_Test(net, train_array, train_ref, test_array, test_ref, batch_size=80, epochs=8)
  train_array, train_ref = 0,0

Start the training
Epoch 1/8
16128/16128 [==============================] - 16s 1ms/step - loss: 0.1894 - acc: 0.9208
Epoch 2/8
16128/16128 [==============================] - 15s 937us/step - loss: 0.1039 - acc: 0.9611
Epoch 3/8
16128/16128 [==============================] - 15s 930us/step - loss: 0.0873 - acc: 0.9676
Epoch 4/8
16128/16128 [==============================] - 15s 920us/step - loss: 0.0805 - acc: 0.9695
Epoch 5/8
16128/16128 [==============================] - 15s 917us/step - loss: 0.0755 - acc: 0.9709
Epoch 6/8
16128/16128 [==============================] - 15s 912us/step - loss: 0.0721 - acc: 0.9721
Epoch 7/8
16128/16128 [==============================] - 15s 913us/step - loss: 0.0678 - acc: 0.9735
Epoch 8/8
576/576 [==============================] - 1s 903us/step
Test loss: 0.17517797317769793
Test accuracy: 94.48835584852431


In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model_a.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_a.h5")
print("Saved model to disk")

#json_file = open('model_b.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)

#loaded_model.compile(loss="categorical_crossentropy", optimizer=optimizer , metrics=['accuracy'])
# load weights into new model
#loaded_model.load_weights("model_b.h5")
#print("Loaded model from disk")

Saved model to disk


In [0]:
  # get test predictions
  predict_test = net.predict(test_array, verbose=1) 
  predict_test = predict_test.argmax(axis=-1)

576/576 [==============================] - 1s 1ms/step


In [0]:

  # mosaicking back
  dim1 = (1584//patches_size)*patches_size
  dim2 = (1584//patches_size)*patches_size
  DIM = [dim1,dim2]
  img_reconstruct = unblockshaped(predict_test, DIM[0], DIM[1])
  img_reconstruct = img_reconstruct.astype('uint8')

  img_ref = unblockshaped(test_ref, DIM[0], DIM[1])
  img_ref = img_ref.astype('uint8')

  # save label image
  save_rgb = gray2rgb(img_reconstruct)
  cv2.imwrite('./weed_result1_' + str(patches_size) + '.png',save_rgb) 

  img_reconstruct = img_reconstruct.reshape(DIM[0]*DIM[1])
  img_ref = img_ref.reshape(DIM[0]*DIM[1])
  
  # get metrics
  accuracy, f1score, recall, precision = compute_metrics(img_ref, img_reconstruct)
  
  print('Test accuracy:', accuracy)
  print('Test f1score:', f1score)
  print('Test recall:', recall)
  print('Test prescision:', precision) 

In [0]:
save_rgb[2]

array([[255,   0,   0],
       [255,   0,   0],
       [255,   0,   0],
       ...,
       [255,   0,   0],
       [255,   0,   0],
       [255,   0,   0]], dtype=uint8)

In [0]:
colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
plt.figure(figsize=(20,16))
plt.subplot(2,3,1)
plt.title("result")
plt.imshow(save_rgb)

img1 = load_img(test_array_path)
img2=load_img(test_reference_path)
plt.subplot(2,3,2)
plt.title("label")
plt.imshow(img2)


plt.subplot(2,3,3)
plt.title("image")
plt.imshow(img1)


In [0]:
img_rr=img_to_array(img1)
img_rr

In [0]:
color=save_rgb